imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path
from PIL import Image

benkaddour architecture


Training iteration 40000 - 80000 epochs

Dropout probability (keep dropout) 0.8

Dropout probability (ratsige dropout) 0.2

learning rate 0.00001

Activation function ReLU

Number of hidden units 512/128

Number of layers (Convolution-Pooling) 4

Classification function for age estimation Softmax

Classication function for gender Sigmoid

In [2]:
class BenkaddourDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [3]:
class BenkaddourCNN(nn.Module):
    def __init__(self):
        super(BenkaddourCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 7))

        self.fc1 = nn.Linear(128 * 6 * 7, 5376)
        self.fc2 = nn.Linear(5376, 512)
        self.fc3 = nn.Linear(512, 128)
        self.dropout = nn.Dropout(p=0.2)
        self.output = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        x = self.dropout(self.fc3(x))
        x = torch.sigmoid(self.output(x))
        return x

dataloaders

In [4]:
def load_folds_dataset(image_root, fold_dir, fold_files):
    image_paths = []
    labels = []

    for fold_file in fold_files:
        with open(os.path.join(fold_dir, fold_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 2:
                    continue  
                name_id, gender = parts[0], parts[4]
                label = 1 if gender.lower() == 'm' else 0
                person_dir = os.path.join(image_root, name_id)

                if not os.path.isdir(person_dir):
                    continue

                for img_file in os.listdir(person_dir):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        image_paths.append(os.path.join(person_dir, img_file))
                        labels.append(label)

    return image_paths, labels


In [5]:
CURRENT_DIR = os.getcwd()
MAIN_FOLDER = Path(CURRENT_DIR).parent
OUTPUT_FOLDER = os.path.join(MAIN_FOLDER, 'aligned')  
FOLD_DATA = os.path.join(MAIN_FOLDER, 'fold_data')    

# Transforms
transform = transforms.Compose([
    transforms.Resize((92, 112)),
    transforms.ToTensor(),
])

model

In [6]:
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return acc, prec, rec, f1


def evaluate(model, dataloader, device):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device).cpu().numpy()

            outputs = model(images)
            predictions = (outputs.cpu().numpy() > 0.5).astype(int)

            y_true.extend(labels)
            y_pred.extend(predictions)

    return compute_metrics(y_true, y_pred)

In [7]:
# training loop
all_folds = [f'fold_{i}_data.txt' for i in range(0, 5)]
test_fold = 'fold_4_data.txt'
cv_folds = [f for f in all_folds if f != test_fold]

epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for i in range(4):
    val_fold = cv_folds[i]
    train_folds = [f for j, f in enumerate(cv_folds) if j != i]

    print(f"Fold {i+1}/4 — Validation: {val_fold}, Train on: {train_folds}, Test on: {test_fold}")

    # Load data
    train_paths, train_labels = load_folds_dataset(OUTPUT_FOLDER, FOLD_DATA, train_folds)
    val_paths, val_labels = load_folds_dataset(OUTPUT_FOLDER, FOLD_DATA, [val_fold])

    train_dataset = BenkaddourDataset(train_paths, train_labels, transform)
    val_dataset = BenkaddourDataset(val_paths, val_labels, transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = BenkaddourCNN().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        y_train_true, y_train_pred = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            y_train_true.extend(labels.cpu().numpy())
            y_train_pred.extend((outputs.detach().cpu().numpy() > 0.5).astype(int))

        # Log training metrics
        train_acc, train_prec, train_rec, train_f1 = compute_metrics(y_train_true, y_train_pred)

        # Evaluate on val
        val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Loss: {running_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | "
              f"Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}")

Fold 1/4 — Validation: fold_0_data.txt, Train on: ['fold_1_data.txt', 'fold_2_data.txt', 'fold_3_data.txt'], Test on: fold_4_data.txt


test evaluation

In [ ]:
# Final evaluation on unseen test set (fold_4.txt)
print("Test Evaluation")

test_paths, test_labels = load_folds_dataset(OUTPUT_FOLDER, FOLD_DATA, [test_fold])
test_dataset = BenkaddourDataset(test_paths, test_labels, transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_loader, device)
print(f"Test Accuracy : {test_acc:.4f}")
print(f"Test Precision: {test_prec:.4f}")
print(f"Test Recall   : {test_rec:.4f}")
print(f"Test F1 Score : {test_f1:.4f}")